In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.1
    Uninstalling google-auth-httplib2-0.1.1:
      Successfully uninstalled google-auth-httplib2-0.1.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0


In [ ]:
pip install --upgrade --quiet langchain-google-community[drive]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import GoogleDriveLoader
import os
from langchain_google_community import GoogleDriveLoader
from langchain_community.document_loaders import UnstructuredFileIOLoader

In [ ]:
GOOGLE_APPLICATION_CREDENTIALS=""
from getpass import getpass
DEEPINFRA_API_TOKEN = getpass()

··········


# Google Authentication

In [45]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')


# Setting up the llm and taking in the google doc

In [43]:
os.environ["DEEPINFRA_API_TOKEN"] = DEEPINFRA_API_TOKEN
from langchain_community.llms import DeepInfra
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]

llm = DeepInfra(model_id="meta-llama/Meta-Llama-3-70B-Instruct")
llm.model_kwargs = {
    "temperature": 0.5,
    "repetition_penalty": 1.2,
    "max_new_tokens": 250,
    "top_p": 0.9,
}
doc_id = "" #Put your Doc ID here
loader = GoogleDriveLoader(
    document_ids=[doc_id],
    credentials_path = "/content/credentials.json",
)


In [46]:
docs=loader.load()

# Summarizes the document

In [49]:
from langchain.chains.summarize import load_summarize_chain


In [50]:
chain =load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

' \nZayaan is a triple major student at Waterloo University passionate about machine learning & statistics, with technical skills & experience in competitive programming, data science internships, research projects, strong theoretical grounding, ability to work under pressure & enthusiasm for feedback.\n\n\nPlease let me know if it meets the requirements!'

# You can ask whatever queries you want

In [51]:
from langchain.chains.question_answering import load_qa_chain

In [57]:
query="" #input your query here

In [58]:
chain=load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)

' Yes, based on his background in CS, Pure Math, and Combinatorics & Optimization, his interest in Machine Learning, and his previous experiences in coding and research, I think he could be a good fit for this internship position. He seems to possess the required qualifications mentioned in the job description, such as strong fundamentals in computer science, proficiency in Python, and experience with AI/ML libraries and frameworks (specifically mentioning PyTorch). Additionally, his passion for understanding theoretical concepts before applying them practically aligns with the responsibilities listed, which involve designing, developing, testing, and deploying AI models.'